In [1]:
#If you need a new visdom server for visualization, run this command in a terminal
# python -m visdom.server
#then navigate here http://localhost:8097

In [5]:

import time
from data import CreateDataLoader
from models import create_model
from util.visualizer import Visualizer
import os


In [1]:
opt={#BASE OPTIONS
    #path to images (should have subfolders trainA, trainB, valA, valB, etc)
    "dataroot":"./datasets/prostate_segmentation/",

    
    #'name of the experiment. It decides where to store samples and models
    "name":"prostate_cycle_gan",
    
    #gpu ids: e.g. 0  0,1,2, 0,2. use -1 for CPU
    "gpu_ids":[0,1],
    
    # models are saved here
    
    "checkpoints_dir":"./checkpoints",
    
    # chooses which model to use. [cycle_gan | pix2pix | test | colorization]
    "model":"pix2pix",
    # instance normalization or batch normalization [instance | batch | none]
    "norm":"instance",
    'use_dropout':True,
    'data_type':16,
    'verbose':True,
    'batchSize':1,
    'which_direction':'AtoB',
    'loadSize':286,
    'fineSize':256,
    'label_nc':35,
    'input_nc':3,
    'output_nc':1,
    'ngf':64,
    'ndf':64,
    'which_model_netD':'basic',
    'which_model_netG':'unet_256',
    'n_layers_D':3,
    'dataset_mode':'aligned',
    'nThreads':8,
    'serial_batches':True,
    'display_winsize':256,
    'display_id':1,
    'display_server':'http://localhost',
    "display_env":"main",
    'display_port':8097,
    'no_dropout':False,
    'max_dataset_size':float("inf"),
    'resize_or_crop':'resize_and_crop',
    'no_flip':True,
    'init_type':'kaiming',
    'init_gain':0.02,
    'verbose':True,
    'suffix':'',
    #TRAIN OPTIONS
    'isTrain':True,
    'display_freq':100,
    'display_ncols':4,
    'update_html_freq':1000,
    'print_freq':100,
    'save_latest_freq':5000,
    'save_epoch_freq':5,
    'continue_train':False,
    'epoch_count':1,
    'phase':'train',
    'which_epoch':'200',
    'niter':100,
    'niter_decay':100,
    'beta1':0.5,
    'lr':0.0002,
    'no_lsgan':True,
    'pool_size':50,
    'no_html':True,
    'lr_policy':'lambda',
    'lr_decay_iters':1,
    'lambda_identity':0.5,
    'lambda_B':10.0,
    'lambda_A':10.0,
    'lambda_L1':50.0
    }
class Map(dict):
    """
    Example:
    m = Map({'first_name': 'Eduardo'}, last_name='Pool', age=24, sports=['Soccer'])
    """
    def __init__(self, *args, **kwargs):
        super(Map, self).__init__(*args, **kwargs)
        for arg in args:
            if isinstance(arg, dict):
                for k, v in arg.items():
                    self[k] = v

        if kwargs:
            for k, v in kwargs.items():
                self[k] = v

    def __getattr__(self, attr):
        return self.get(attr)

    def __setattr__(self, key, value):
        self.__setitem__(key, value)

    def __setitem__(self, key, value):
        super(Map, self).__setitem__(key, value)
        self.__dict__.update({key: value})

    def __delattr__(self, item):
        self.__delitem__(item)

    def __delitem__(self, key):
        super(Map, self).__delitem__(key)
        del self.__dict__[key]
opt = Map(opt)

In [6]:
def train(opt):
    data_loader = CreateDataLoader(opt)
    dataset = data_loader.load_data()
    dataset_size = len(data_loader)
    print('The number of training images = %d' % dataset_size)

    model = create_model(opt)      # create a model given opt.model and other options
    model.setup(opt)               # regular setup: load and print networks; create schedulers
    visualizer = Visualizer(opt)   # create a visualizer that display/save images and plots
    total_steps = 0                # the total number of training iterations

    for epoch in range(opt.epoch_count, opt.niter + opt.niter_decay + 1):
        epoch_start_time = time.time()
        iter_data_time = time.time()
        epoch_iter = 0

        for i, data in enumerate(dataset):
            iter_start_time = time.time()
            if total_steps % opt.print_freq == 0:
                t_data = iter_start_time - iter_data_time
            visualizer.reset()
            total_steps += opt.batchSize
            epoch_iter += opt.batchSize
            model.set_input(data)
            model.optimize_parameters()

            if total_steps % opt.display_freq == 0:
                save_result = total_steps % opt.update_html_freq == 0
                visualizer.display_current_results(model.get_current_visuals(), epoch, save_result)

            if total_steps % opt.print_freq == 0:
                losses = model.get_current_losses()
                t = (time.time() - iter_start_time) / opt.batchSize
                visualizer.print_current_losses(epoch, epoch_iter, losses, t, t_data)
                if opt.display_id > 0:
                    visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, opt, losses)

            if total_steps % opt.save_latest_freq == 0:
                print('saving the latest model (epoch %d, total_steps %d)' %
                      (epoch, total_steps))
                model.save_networks('latest')

            iter_data_time = time.time()
        if epoch % opt.save_epoch_freq == 0:
            print('saving the model at the end of epoch %d, iters %d' %
                  (epoch, total_steps))
            model.save_networks('latest')
            model.save_networks(epoch)

        print('End of epoch %d / %d \t Time Taken: %d sec' %
              (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))
        model.update_learning_rate()

In [7]:
droupouts = [0,0.1,0.25,0.4]
for drop_out in droupouts:
        new_opt = Map(opt)
        new_opt.drop_out
        new_opt.name = 'prostate_cyc_batch_'+'_lr_'+str(learning_rate)+'_batch'+str(batch_size)
        if not os.path.exists(os.path.join(new_opt.checkpoints_dir,new_opt.name)):
            os.mkdir(os.path.join(new_opt.checkpoints_dir,new_opt.name))
        train(new_opt)

dataset [AlignedDataset] was created
The number of training images = 2280


initialize network with kaiming
initialize network with kaiming
model [Pix2PixModel] was created
---------- Networks initialized -------------
DataParallel(
  (module): UnetGenerator(
    (model): UnetSkipConnectionBlock(
      (model): Sequential(
        (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): UnetSkipConnectionBlock(
          (model): Sequential(
            (0): LeakyReLU(negative_slope=0.2, inplace)
            (1): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
            (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): UnetSkipConnectionBlock(
              (model): Sequential(
                (0): LeakyReLU(negative_slope=0.2, inplace)
                (1): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

KeyboardInterrupt: 

In [8]:
experiments = [(4,8*0.0002)]
for batch_size,learning_rate in experiments[::-1]:
        new_opt = Map(opt)
        new_opt.batchSize = batch_size
        new_opt.lr = learning_rate
        new_opt.name = 'prostate_cyc_batch_'+'_lr_'+str(learning_rate)+'_batch'+str(batch_size)
        if not os.path.exists(os.path.join(new_opt.checkpoints_dir,new_opt.name)):
            os.mkdir(os.path.join(new_opt.checkpoints_dir,new_opt.name))
        train(new_opt)

dataset [AlignedDataset] was created
The number of training images = 2280


initialize network with kaiming
initialize network with kaiming
model [Pix2PixModel] was created
---------- Networks initialized -------------
DataParallel(
  (module): UnetGenerator(
    (model): UnetSkipConnectionBlock(
      (model): Sequential(
        (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): UnetSkipConnectionBlock(
          (model): Sequential(
            (0): LeakyReLU(negative_slope=0.2, inplace)
            (1): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
            (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): UnetSkipConnectionBlock(
              (model): Sequential(
                (0): LeakyReLU(negative_slope=0.2, inplace)
                (1): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

(epoch: 1, iters: 2000, time: 0.075, data: 0.001) G_GAN: 0.768 G_L1: 0.000 D_real: 0.778 D_fake: 0.616 
(epoch: 1, iters: 2100, time: 0.072, data: 0.002) G_GAN: 0.699 G_L1: 0.000 D_real: 0.717 D_fake: 0.671 
(epoch: 1, iters: 2200, time: 0.077, data: 0.001) G_GAN: 0.789 G_L1: 1.586 D_real: 0.638 D_fake: 0.631 
End of epoch 1 / 200 	 Time Taken: 110 sec
learning rate = 0.0016000
(epoch: 2, iters: 20, time: 0.072, data: 0.002) G_GAN: 0.737 G_L1: 0.001 D_real: 0.735 D_fake: 0.649 
(epoch: 2, iters: 120, time: 0.073, data: 0.001) G_GAN: 0.725 G_L1: 0.002 D_real: 0.735 D_fake: 0.653 
(epoch: 2, iters: 220, time: 0.071, data: 0.001) G_GAN: 0.726 G_L1: 2.749 D_real: 0.570 D_fake: 0.678 
(epoch: 2, iters: 320, time: 0.072, data: 0.001) G_GAN: 0.747 G_L1: 0.000 D_real: 0.751 D_fake: 0.639 
(epoch: 2, iters: 420, time: 0.074, data: 0.001) G_GAN: 0.719 G_L1: 0.001 D_real: 0.728 D_fake: 0.660 
(epoch: 2, iters: 520, time: 0.074, data: 0.002) G_GAN: 0.736 G_L1: 1.419 D_real: 0.614 D_fake: 0.661 
(e

(epoch: 5, iters: 480, time: 0.073, data: 0.002) G_GAN: 0.690 G_L1: 0.000 D_real: 0.711 D_fake: 0.677 
(epoch: 5, iters: 580, time: 0.072, data: 0.002) G_GAN: 0.780 G_L1: 3.522 D_real: 0.586 D_fake: 0.655 
(epoch: 5, iters: 680, time: 0.071, data: 0.002) G_GAN: 0.751 G_L1: 0.000 D_real: 0.775 D_fake: 0.622 
(epoch: 5, iters: 780, time: 0.075, data: 0.002) G_GAN: 0.681 G_L1: 0.000 D_real: 0.708 D_fake: 0.680 
(epoch: 5, iters: 880, time: 0.070, data: 0.002) G_GAN: 0.726 G_L1: 2.562 D_real: 0.570 D_fake: 0.682 
saving the latest model (epoch 5, total_steps 10000)
(epoch: 5, iters: 980, time: 0.073, data: 0.002) G_GAN: 0.742 G_L1: 0.000 D_real: 0.765 D_fake: 0.626 
(epoch: 5, iters: 1080, time: 0.075, data: 0.001) G_GAN: 0.679 G_L1: 0.000 D_real: 0.685 D_fake: 0.704 
(epoch: 5, iters: 1180, time: 0.074, data: 0.002) G_GAN: 0.781 G_L1: 2.010 D_real: 0.614 D_fake: 0.654 
(epoch: 5, iters: 1280, time: 0.072, data: 0.002) G_GAN: 0.756 G_L1: 0.000 D_real: 0.778 D_fake: 0.621 
(epoch: 5, iters:

(epoch: 8, iters: 1240, time: 0.073, data: 0.002) G_GAN: 0.745 G_L1: 2.386 D_real: 0.593 D_fake: 0.655 
(epoch: 8, iters: 1340, time: 0.070, data: 0.002) G_GAN: 0.742 G_L1: 0.000 D_real: 0.746 D_fake: 0.644 
(epoch: 8, iters: 1440, time: 0.076, data: 0.002) G_GAN: 0.724 G_L1: 0.000 D_real: 0.732 D_fake: 0.657 
(epoch: 8, iters: 1540, time: 0.075, data: 0.002) G_GAN: 0.736 G_L1: 1.488 D_real: 0.603 D_fake: 0.660 
(epoch: 8, iters: 1640, time: 0.073, data: 0.003) G_GAN: 0.748 G_L1: 0.000 D_real: 0.754 D_fake: 0.638 
(epoch: 8, iters: 1740, time: 0.074, data: 0.002) G_GAN: 0.730 G_L1: 0.004 D_real: 0.740 D_fake: 0.649 
(epoch: 8, iters: 1840, time: 0.075, data: 0.002) G_GAN: 0.738 G_L1: 0.842 D_real: 0.635 D_fake: 0.655 
(epoch: 8, iters: 1940, time: 0.072, data: 0.002) G_GAN: 0.752 G_L1: 0.000 D_real: 0.755 D_fake: 0.635 
(epoch: 8, iters: 2040, time: 0.073, data: 0.002) G_GAN: 0.721 G_L1: 0.000 D_real: 0.730 D_fake: 0.664 
(epoch: 8, iters: 2140, time: 0.073, data: 0.002) G_GAN: 0.735 G

...

(epoch: 186, iters: 1400, time: 0.072, data: 0.002) G_GAN: 0.778 G_L1: 0.000 D_real: 0.778 D_fake: 0.647 
(epoch: 186, iters: 1500, time: 0.074, data: 0.002) G_GAN: 0.724 G_L1: 0.000 D_real: 0.729 D_fake: 0.662 
(epoch: 186, iters: 1600, time: 0.071, data: 0.002) G_GAN: 1.078 G_L1: 2.124 D_real: 0.548 D_fake: 0.575 
(epoch: 186, iters: 1700, time: 0.073, data: 0.002) G_GAN: 0.746 G_L1: 0.000 D_real: 0.749 D_fake: 0.625 
(epoch: 186, iters: 1800, time: 0.075, data: 0.002) G_GAN: 0.725 G_L1: 0.000 D_real: 0.728 D_fake: 0.666 
(epoch: 186, iters: 1900, time: 0.076, data: 0.003) G_GAN: 0.921 G_L1: 3.702 D_real: 0.504 D_fake: 0.652 
(epoch: 186, iters: 2000, time: 0.073, data: 0.003) G_GAN: 0.736 G_L1: 0.000 D_real: 0.738 D_fake: 0.642 
(epoch: 186, iters: 2100, time: 0.074, data: 0.002) G_GAN: 0.688 G_L1: 0.000 D_real: 0.696 D_fake: 0.668 
(epoch: 186, iters: 2200, time: 0.073, data: 0.002) G_GAN: 0.967 G_L1: 1.583 D_real: 0.580 D_fake: 0.606 
End of epoch 186 / 200 	 Time Taken: 110 sec
l

(epoch: 189, iters: 2060, time: 0.074, data: 0.003) G_GAN: 0.744 G_L1: 0.000 D_real: 0.750 D_fake: 0.631 
(epoch: 189, iters: 2160, time: 0.072, data: 0.002) G_GAN: 0.720 G_L1: 0.000 D_real: 0.725 D_fake: 0.646 
(epoch: 189, iters: 2260, time: 0.072, data: 0.003) G_GAN: 1.085 G_L1: 2.749 D_real: 0.521 D_fake: 0.597 
End of epoch 189 / 200 	 Time Taken: 110 sec
learning rate = 0.0001743
(epoch: 190, iters: 80, time: 0.077, data: 0.002) G_GAN: 0.732 G_L1: 0.000 D_real: 0.736 D_fake: 0.605 
(epoch: 190, iters: 180, time: 0.071, data: 0.002) G_GAN: 0.733 G_L1: 0.000 D_real: 0.735 D_fake: 0.637 
(epoch: 190, iters: 280, time: 0.075, data: 0.003) G_GAN: 1.055 G_L1: 1.026 D_real: 0.600 D_fake: 0.581 
(epoch: 190, iters: 380, time: 0.073, data: 0.003) G_GAN: 0.761 G_L1: 0.000 D_real: 0.767 D_fake: 0.650 
(epoch: 190, iters: 480, time: 0.072, data: 0.003) G_GAN: 0.713 G_L1: 0.000 D_real: 0.714 D_fake: 0.635 
(epoch: 190, iters: 580, time: 0.073, data: 0.003) G_GAN: 1.149 G_L1: 3.541 D_real: 0.5

(epoch: 193, iters: 340, time: 0.073, data: 0.002) G_GAN: 0.902 G_L1: 1.290 D_real: 0.578 D_fake: 0.606 
(epoch: 193, iters: 440, time: 0.071, data: 0.001) G_GAN: 0.714 G_L1: 0.000 D_real: 0.715 D_fake: 0.677 
(epoch: 193, iters: 540, time: 0.070, data: 0.001) G_GAN: 0.720 G_L1: 0.000 D_real: 0.726 D_fake: 0.645 
(epoch: 193, iters: 640, time: 0.072, data: 0.002) G_GAN: 0.998 G_L1: 2.682 D_real: 0.538 D_fake: 0.587 
(epoch: 193, iters: 740, time: 0.072, data: 0.001) G_GAN: 0.721 G_L1: 0.000 D_real: 0.723 D_fake: 0.659 
(epoch: 193, iters: 840, time: 0.071, data: 0.004) G_GAN: 0.702 G_L1: 0.000 D_real: 0.706 D_fake: 0.667 
(epoch: 193, iters: 940, time: 0.072, data: 0.003) G_GAN: 1.195 G_L1: 3.230 D_real: 0.527 D_fake: 0.592 
(epoch: 193, iters: 1040, time: 0.072, data: 0.003) G_GAN: 0.756 G_L1: 0.000 D_real: 0.761 D_fake: 0.655 
(epoch: 193, iters: 1140, time: 0.071, data: 0.002) G_GAN: 0.716 G_L1: 0.000 D_real: 0.717 D_fake: 0.676 
(epoch: 193, iters: 1240, time: 0.074, data: 0.002) G

(epoch: 196, iters: 1000, time: 0.072, data: 0.002) G_GAN: 1.138 G_L1: 1.678 D_real: 0.591 D_fake: 0.593 
(epoch: 196, iters: 1100, time: 0.076, data: 0.002) G_GAN: 0.711 G_L1: 0.000 D_real: 0.716 D_fake: 0.647 
(epoch: 196, iters: 1200, time: 0.073, data: 0.003) G_GAN: 0.732 G_L1: 0.000 D_real: 0.734 D_fake: 0.635 
(epoch: 196, iters: 1300, time: 0.072, data: 0.002) G_GAN: 1.053 G_L1: 1.744 D_real: 0.577 D_fake: 0.573 
(epoch: 196, iters: 1400, time: 0.072, data: 0.003) G_GAN: 0.766 G_L1: 0.000 D_real: 0.766 D_fake: 0.654 
(epoch: 196, iters: 1500, time: 0.072, data: 0.003) G_GAN: 0.719 G_L1: 0.000 D_real: 0.724 D_fake: 0.595 
(epoch: 196, iters: 1600, time: 0.074, data: 0.002) G_GAN: 1.115 G_L1: 2.124 D_real: 0.548 D_fake: 0.593 
(epoch: 196, iters: 1700, time: 0.073, data: 0.002) G_GAN: 0.740 G_L1: 0.000 D_real: 0.740 D_fake: 0.657 
(epoch: 196, iters: 1800, time: 0.072, data: 0.002) G_GAN: 0.734 G_L1: 0.000 D_real: 0.736 D_fake: 0.625 
(epoch: 196, iters: 1900, time: 0.073, data: 0

(epoch: 199, iters: 1760, time: 0.071, data: 0.002) G_GAN: 0.724 G_L1: 0.000 D_real: 0.725 D_fake: 0.627 
(epoch: 199, iters: 1860, time: 0.070, data: 0.002) G_GAN: 0.776 G_L1: 0.000 D_real: 0.777 D_fake: 0.645 
(epoch: 199, iters: 1960, time: 0.075, data: 0.003) G_GAN: 0.919 G_L1: 2.015 D_real: 0.553 D_fake: 0.598 
(epoch: 199, iters: 2060, time: 0.072, data: 0.003) G_GAN: 0.732 G_L1: 0.000 D_real: 0.734 D_fake: 0.632 
(epoch: 199, iters: 2160, time: 0.072, data: 0.002) G_GAN: 0.721 G_L1: 0.000 D_real: 0.722 D_fake: 0.656 
(epoch: 199, iters: 2260, time: 0.074, data: 0.002) G_GAN: 0.995 G_L1: 2.749 D_real: 0.506 D_fake: 0.604 
End of epoch 199 / 200 	 Time Taken: 110 sec
learning rate = 0.0000158
(epoch: 200, iters: 80, time: 0.072, data: 0.002) G_GAN: 0.723 G_L1: 0.000 D_real: 0.724 D_fake: 0.670 
(epoch: 200, iters: 180, time: 0.072, data: 0.003) G_GAN: 0.729 G_L1: 0.000 D_real: 0.730 D_fake: 0.646 
(epoch: 200, iters: 280, time: 0.072, data: 0.002) G_GAN: 1.037 G_L1: 1.026 D_real: 

In [ ]:
experiments = [(8,16*0.0002)]
for batch_size,learning_rate in experiments[::-1]:
        new_opt = Map(opt)
        new_opt.batchSize = batch_size
        new_opt.lr = learning_rate
        new_opt.name = 'prostate_cyc_batch_'+'_lr_'+str(learning_rate)+'_batch'+str(batch_size)
        if not os.path.exists(os.path.join(new_opt.checkpoints_dir,new_opt.name)):
            os.mkdir(os.path.join(new_opt.checkpoints_dir,new_opt.name))
        train(new_opt)

In [12]:
init_types = ['normal','kaiming','xavier']
for init_type in init_types:
        new_opt = Map(opt)
        new_opt.init_type = init_type
        new_opt.name = 'prostate_cyc_batch_'+'_init_type_'+str(init_type)
        if not os.path.exists(os.path.join(new_opt.checkpoints_dir,new_opt.name)):
            os.mkdir(os.path.join(new_opt.checkpoints_dir,new_opt.name))
        train(new_opt)

dataset [AlignedDataset] was created
The number of training images = 2280


initialize network with normal
initialize network with normal
model [Pix2PixModel] was created
---------- Networks initialized -------------
DataParallel(
  (module): UnetGenerator(
    (model): UnetSkipConnectionBlock(
      (model): Sequential(
        (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): UnetSkipConnectionBlock(
          (model): Sequential(
            (0): LeakyReLU(negative_slope=0.2, inplace)
            (1): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
            (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): UnetSkipConnectionBlock(
              (model): Sequential(
                (0): LeakyReLU(negative_slope=0.2, inplace)
                (1): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

(epoch: 1, iters: 2000, time: 0.098, data: 0.001) G_GAN: 0.762 G_L1: 0.002 D_real: 0.772 D_fake: 0.620 
(epoch: 1, iters: 2100, time: 0.095, data: 0.001) G_GAN: 0.698 G_L1: 0.000 D_real: 0.673 D_fake: 0.715 
(epoch: 1, iters: 2200, time: 0.107, data: 0.001) G_GAN: 0.768 G_L1: 1.928 D_real: 0.637 D_fake: 0.631 
End of epoch 1 / 200 	 Time Taken: 122 sec
learning rate = 0.0002000
(epoch: 2, iters: 20, time: 0.105, data: 0.000) G_GAN: 0.713 G_L1: 0.017 D_real: 0.720 D_fake: 0.668 
(epoch: 2, iters: 120, time: 0.098, data: 0.001) G_GAN: 0.689 G_L1: 0.012 D_real: 0.683 D_fake: 0.709 
(epoch: 2, iters: 220, time: 0.109, data: 0.002) G_GAN: 0.823 G_L1: 3.204 D_real: 0.649 D_fake: 0.574 
(epoch: 2, iters: 320, time: 0.098, data: 0.002) G_GAN: 0.739 G_L1: 0.027 D_real: 0.752 D_fake: 0.651 
(epoch: 2, iters: 420, time: 0.099, data: 0.001) G_GAN: 0.694 G_L1: 0.001 D_real: 0.696 D_fake: 0.691 
(epoch: 2, iters: 520, time: 0.097, data: 0.001) G_GAN: 0.766 G_L1: 1.603 D_real: 0.634 D_fake: 0.631 
(e

(epoch: 5, iters: 480, time: 0.096, data: 0.001) G_GAN: 0.701 G_L1: 0.000 D_real: 0.701 D_fake: 0.704 
(epoch: 5, iters: 580, time: 0.102, data: 0.001) G_GAN: 0.668 G_L1: 1.060 D_real: 0.643 D_fake: 0.741 
(epoch: 5, iters: 680, time: 0.099, data: 0.001) G_GAN: 0.701 G_L1: 0.000 D_real: 0.702 D_fake: 0.685 
(epoch: 5, iters: 780, time: 0.094, data: 0.001) G_GAN: 0.701 G_L1: 0.000 D_real: 0.701 D_fake: 0.685 
(epoch: 5, iters: 880, time: 0.093, data: 0.001) G_GAN: 0.685 G_L1: 0.634 D_real: 0.687 D_fake: 0.701 
saving the latest model (epoch 5, total_steps 10000)
(epoch: 5, iters: 980, time: 0.097, data: 0.002) G_GAN: 0.680 G_L1: 1.904 D_real: 0.669 D_fake: 0.728 
(epoch: 5, iters: 1080, time: 0.097, data: 0.001) G_GAN: 0.678 G_L1: 0.000 D_real: 0.681 D_fake: 0.696 
(epoch: 5, iters: 1180, time: 0.099, data: 0.001) G_GAN: 0.693 G_L1: 0.424 D_real: 0.693 D_fake: 0.694 
(epoch: 5, iters: 1280, time: 0.098, data: 0.001) G_GAN: 0.701 G_L1: 0.000 D_real: 0.701 D_fake: 0.686 
(epoch: 5, iters:

(epoch: 8, iters: 1240, time: 0.107, data: 0.001) G_GAN: 0.705 G_L1: 0.539 D_real: 0.701 D_fake: 0.700 
(epoch: 8, iters: 1340, time: 0.110, data: 0.001) G_GAN: 0.697 G_L1: 0.000 D_real: 0.697 D_fake: 0.690 
(epoch: 8, iters: 1440, time: 0.099, data: 0.001) G_GAN: 0.701 G_L1: 0.000 D_real: 0.701 D_fake: 0.687 
(epoch: 8, iters: 1540, time: 0.101, data: 0.001) G_GAN: 0.694 G_L1: 0.981 D_real: 0.682 D_fake: 0.692 
(epoch: 8, iters: 1640, time: 0.099, data: 0.001) G_GAN: 0.702 G_L1: 0.000 D_real: 0.702 D_fake: 0.684 
(epoch: 8, iters: 1740, time: 0.101, data: 0.001) G_GAN: 0.702 G_L1: 0.001 D_real: 0.702 D_fake: 0.683 
(epoch: 8, iters: 1840, time: 0.111, data: 0.001) G_GAN: 0.700 G_L1: 0.194 D_real: 0.697 D_fake: 0.686 
(epoch: 8, iters: 1940, time: 0.106, data: 0.001) G_GAN: 0.698 G_L1: 0.000 D_real: 0.455 D_fake: 1.299 
(epoch: 8, iters: 2040, time: 0.097, data: 0.001) G_GAN: 0.700 G_L1: 0.000 D_real: 0.700 D_fake: 0.687 
(epoch: 8, iters: 2140, time: 0.102, data: 0.001) G_GAN: 0.741 G

...

(epoch: 186, iters: 1400, time: 0.097, data: 0.002) G_GAN: 0.794 G_L1: 0.000 D_real: 0.798 D_fake: 0.642 
(epoch: 186, iters: 1500, time: 0.098, data: 0.002) G_GAN: 0.745 G_L1: 0.000 D_real: 0.746 D_fake: 0.573 
(epoch: 186, iters: 1600, time: 0.098, data: 0.001) G_GAN: 0.870 G_L1: 1.704 D_real: 0.569 D_fake: 0.637 
(epoch: 186, iters: 1700, time: 0.099, data: 0.002) G_GAN: 0.767 G_L1: 0.000 D_real: 0.776 D_fake: 0.597 
(epoch: 186, iters: 1800, time: 0.095, data: 0.001) G_GAN: 0.716 G_L1: 0.000 D_real: 0.717 D_fake: 0.675 
(epoch: 186, iters: 1900, time: 0.098, data: 0.001) G_GAN: 0.887 G_L1: 4.005 D_real: 0.506 D_fake: 0.666 
(epoch: 186, iters: 2000, time: 0.097, data: 0.001) G_GAN: 0.781 G_L1: 0.000 D_real: 0.785 D_fake: 0.611 
(epoch: 186, iters: 2100, time: 0.094, data: 0.001) G_GAN: 0.703 G_L1: 0.000 D_real: 0.706 D_fake: 0.688 
(epoch: 186, iters: 2200, time: 0.097, data: 0.001) G_GAN: 0.790 G_L1: 1.935 D_real: 0.523 D_fake: 0.656 
End of epoch 186 / 200 	 Time Taken: 124 sec
l

(epoch: 189, iters: 2060, time: 0.094, data: 0.002) G_GAN: 0.773 G_L1: 0.000 D_real: 0.775 D_fake: 0.636 
(epoch: 189, iters: 2160, time: 0.097, data: 0.002) G_GAN: 0.739 G_L1: 0.000 D_real: 0.743 D_fake: 0.579 
(epoch: 189, iters: 2260, time: 0.095, data: 0.002) G_GAN: 0.917 G_L1: 3.017 D_real: 0.588 D_fake: 0.561 
End of epoch 189 / 200 	 Time Taken: 125 sec
learning rate = 0.0000218
(epoch: 190, iters: 80, time: 0.104, data: 0.002) G_GAN: 0.729 G_L1: 0.000 D_real: 0.729 D_fake: 0.668 
(epoch: 190, iters: 180, time: 0.098, data: 0.001) G_GAN: 0.693 G_L1: 0.000 D_real: 0.693 D_fake: 0.696 
(epoch: 190, iters: 280, time: 0.098, data: 0.002) G_GAN: 0.800 G_L1: 1.052 D_real: 0.596 D_fake: 0.628 
(epoch: 190, iters: 380, time: 0.098, data: 0.002) G_GAN: 0.739 G_L1: 0.000 D_real: 0.742 D_fake: 0.614 
(epoch: 190, iters: 480, time: 0.099, data: 0.002) G_GAN: 0.709 G_L1: 0.000 D_real: 0.709 D_fake: 0.680 
(epoch: 190, iters: 580, time: 0.099, data: 0.002) G_GAN: 0.871 G_L1: 4.208 D_real: 0.5

(epoch: 193, iters: 340, time: 0.109, data: 0.002) G_GAN: 0.752 G_L1: 0.576 D_real: 0.615 D_fake: 0.648 
(epoch: 193, iters: 440, time: 0.111, data: 0.001) G_GAN: 0.743 G_L1: 0.000 D_real: 0.746 D_fake: 0.664 
(epoch: 193, iters: 540, time: 0.094, data: 0.002) G_GAN: 0.693 G_L1: 0.000 D_real: 0.697 D_fake: 0.604 
(epoch: 193, iters: 640, time: 0.094, data: 0.001) G_GAN: 0.907 G_L1: 3.338 D_real: 0.550 D_fake: 0.576 
(epoch: 193, iters: 740, time: 0.105, data: 0.002) G_GAN: 0.747 G_L1: 0.000 D_real: 0.752 D_fake: 0.597 
(epoch: 193, iters: 840, time: 0.095, data: 0.001) G_GAN: 0.714 G_L1: 0.000 D_real: 0.717 D_fake: 0.613 
(epoch: 193, iters: 940, time: 0.109, data: 0.002) G_GAN: 0.847 G_L1: 1.224 D_real: 0.639 D_fake: 0.605 
(epoch: 193, iters: 1040, time: 0.095, data: 0.001) G_GAN: 0.805 G_L1: 0.000 D_real: 0.810 D_fake: 0.607 
(epoch: 193, iters: 1140, time: 0.097, data: 0.001) G_GAN: 0.719 G_L1: 0.000 D_real: 0.720 D_fake: 0.534 
(epoch: 193, iters: 1240, time: 0.096, data: 0.001) G

(epoch: 196, iters: 1000, time: 0.100, data: 0.002) G_GAN: 0.920 G_L1: 2.064 D_real: 0.580 D_fake: 0.636 
(epoch: 196, iters: 1100, time: 0.099, data: 0.002) G_GAN: 0.727 G_L1: 0.000 D_real: 0.727 D_fake: 0.671 
(epoch: 196, iters: 1200, time: 0.098, data: 0.002) G_GAN: 0.726 G_L1: 0.000 D_real: 0.726 D_fake: 0.664 
(epoch: 196, iters: 1300, time: 0.098, data: 0.001) G_GAN: 0.949 G_L1: 3.100 D_real: 0.566 D_fake: 0.699 
(epoch: 196, iters: 1400, time: 0.098, data: 0.002) G_GAN: 0.795 G_L1: 0.000 D_real: 0.796 D_fake: 0.560 
(epoch: 196, iters: 1500, time: 0.107, data: 0.001) G_GAN: 0.759 G_L1: 0.000 D_real: 0.760 D_fake: 0.641 
(epoch: 196, iters: 1600, time: 0.099, data: 0.002) G_GAN: 0.907 G_L1: 1.704 D_real: 0.563 D_fake: 0.591 
(epoch: 196, iters: 1700, time: 0.109, data: 0.001) G_GAN: 0.759 G_L1: 0.000 D_real: 0.763 D_fake: 0.585 
(epoch: 196, iters: 1800, time: 0.095, data: 0.001) G_GAN: 0.720 G_L1: 0.000 D_real: 0.721 D_fake: 0.622 
(epoch: 196, iters: 1900, time: 0.097, data: 0

(epoch: 199, iters: 1760, time: 0.098, data: 0.002) G_GAN: 0.800 G_L1: 0.000 D_real: 0.801 D_fake: 0.613 
(epoch: 199, iters: 1860, time: 0.099, data: 0.001) G_GAN: 0.738 G_L1: 0.000 D_real: 0.738 D_fake: 0.660 
(epoch: 199, iters: 1960, time: 0.098, data: 0.002) G_GAN: 0.819 G_L1: 0.417 D_real: 0.666 D_fake: 0.606 
(epoch: 199, iters: 2060, time: 0.095, data: 0.002) G_GAN: 0.790 G_L1: 0.000 D_real: 0.790 D_fake: 0.628 
(epoch: 199, iters: 2160, time: 0.097, data: 0.002) G_GAN: 0.741 G_L1: 0.000 D_real: 0.741 D_fake: 0.586 
(epoch: 199, iters: 2260, time: 0.096, data: 0.002) G_GAN: 0.873 G_L1: 3.017 D_real: 0.522 D_fake: 0.606 
End of epoch 199 / 200 	 Time Taken: 123 sec
learning rate = 0.0000020
(epoch: 200, iters: 80, time: 0.107, data: 0.002) G_GAN: 0.728 G_L1: 0.000 D_real: 0.728 D_fake: 0.668 
(epoch: 200, iters: 180, time: 0.110, data: 0.001) G_GAN: 0.719 G_L1: 0.000 D_real: 0.719 D_fake: 0.673 
(epoch: 200, iters: 280, time: 0.096, data: 0.002) G_GAN: 0.811 G_L1: 1.052 D_real: 